In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("gdrive/MyDrive/aYin/alaska")

Mounted at /content/gdrive


In [ ]:
!pip install bertopic
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy]
!pip install bertopic[use]

In [ ]:
!pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
import string
from string import digits
#import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import gensim
from gensim.utils import simple_preprocess
import re


stop_words = stopwords.words('english')

def my_preprocessor(doc):
  doc_lower = doc.lower().replace("amp", "")
  doc_lower = re.sub(r'http\S+', '', doc_lower)
  doc_lower = re.sub(r'@\S+', '', doc_lower)
  remove_digits = str.maketrans('', '', string.digits)
  translator = str.maketrans('', '', string.punctuation)
  doc_no_punctuation = doc_lower.translate(translator).translate(remove_digits)
  return(doc_no_punctuation)

import spacy
spacy.load('en_core_web_sm')
lemmatizer = spacy.lang.en.English()

def my_tokenizer1(doc):
  tokens = lemmatizer(doc)
  return([token.lemma_ for token in tokens])

def my_tokenizer2(doc):
  if len(doc) == 0:
    return('')
  else:
    doc_tokens =gensim.utils.simple_preprocess(doc, deacc=True)
    ps = PorterStemmer()
    doc_filtered = [w for w in doc_tokens if not w in stop_words]
    doc_stem = [ps.stem(w) for w in doc_filtered]
    # return(doc_stem)
    return (" ".join(doc_stem))

In [ ]:
docs = pd.read_csv("alaska_all_sub_class.tsv", sep ='\t')
print(len(docs))
docs['text'] = docs['text'].apply(my_preprocessor)
docs['text'].replace('', np.nan, inplace=True)
docs.dropna(subset=['text'], inplace=True)
docs['text'] = docs['text'].apply(my_tokenizer2)
docs['text'].replace('', np.nan, inplace=True)
docs.dropna(subset=['text'], inplace=True)
print(len(docs))
docs.head()

data = docs['text'].tolist()
# year_month = docs['year_month'].tolist()

23685
23677


In [ ]:
#import hdbscan
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
#import umap
from umap import UMAP

# Prepare custom models
# min_sample : large number leads to more outliers
#cluster_selection_epsilon
hdbscan_model = HDBSCAN(min_cluster_size=20,min_samples=5, metric='euclidean',
                        cluster_selection_method='eom', prediction_data=True)

# random state for reproducibility
#n_neighbors is the numer of neighboring sample points used when making the manifold approximation.
#Increasing this value typically results in a more global view of the embedding structure whilst smaller values result in a more local view.
#Increasing this value often results in larger clusters being created.
umap_model = UMAP(n_neighbors=15, n_components=10,random_state=120,min_dist=0.0, metric='cosine')
vectorizer_model = CountVectorizer(ngram_range=(2, 3),max_df=0.95, min_df=0.05)
# vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       vectorizer_model=vectorizer_model)

topics, probs = topic_model.fit_transform(data)

In [ ]:
fig =topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)
# fig = topic_model.visualize_barchart(top_n_topics=12, n_words=20, width=350)

# fig.write_image("forest_landscape_topic.png")
# fig.write_image("landscape_topic.png")
topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)

In [ ]:
# topic_model.reduce_topics(data, nr_topics="auto")
fig = topic_model.visualize_barchart(top_n_topics=8, width=400, height=512, n_words=20)
fig.write_image("landscape_topic.png")
# frequency = topic_model.get_topic_freq()
# topic_model.get_topic_freq()
# # print(frequency)
topic_model.visualize_barchart(top_n_topics=8, width=512, height=512, n_words=20)
# topic_model.visualize_barchart(top_n_topics=12, width=512, height=512)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig.update_layout(title_font_size=28)
fig.update_layout(xaxis_title_font_size=100, yaxis_title_font_size=100)
fig.update_layout(font=dict(
        size=17,
        color="Black"

    ))
fig.write_image("climate_topic.png")



In [ ]:
# fig = topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)

# fig.write_html("file.html")

# fig = topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)
fig = topic_model.visualize_barchart(top_n_topics=12, n_words=20, width=350)

# fig.write_image("forest_landscape_topic.png")
fig.write_image("landscape_topic.png")
# fig.write_image("ecosystem_topic.png")
# fig.write_image("ecological_topic.png")

In [ ]:
# dir(topic_model)
# #topic_model.get_topics()
# topic_model.get_topic_freq()
# topic_model.get_topic_freq().head(50)

In [ ]:
# # topics_over_time = topic_model.topics_over_time(data, year_month, global_tuning=True, evolution_tuning=True, nr_bins=20)
# topic_model.topics_over_time(data, year_month, nr_bins=84)

In [ ]:
import pandas as pd
from typing import List, Union
import plotly.graph_objects as go
from sklearn.preprocessing import normalize
from plotly.validators.scatter.marker import SymbolValidator



def visualize_topics_over_time(topic_model,
                               topics_over_time: pd.DataFrame,
                               top_n_topics: int = None,
                               topics: List[int] = None,
                               normalize_frequency: bool = False,
                               custom_labels: Union[bool, str] = False,
                               title: str = "<b>Topics over Time</b>",
                               width: int = 1300,
                               height: int = 450) -> go.Figure:
    """ Visualize topics over time
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics_over_time: The topics you would like to be visualized with the
                          corresponding topic representation
        top_n_topics: To visualize the most frequent topics instead of all
        topics: Select which topics you would like to be visualized
        normalize_frequency: Whether to normalize each topic's frequency individually
        custom_labels: If bool, whether to use custom topic labels that were defined using
                       `topic_model.set_topic_labels`.
                       If `str`, it uses labels from other aspects, e.g., "Aspect1".
        title: Title of the plot.
        width: The width of the figure.
        height: The height of the figure.
    Returns:
        A plotly.graph_objects.Figure including all traces
    Examples:
    To visualize the topics over time, simply run:
    ```python
    topics_over_time = topic_model.topics_over_time(docs, timestamps)
    topic_model.visualize_topics_over_time(topics_over_time)
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_topics_over_time(topics_over_time)
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/trump.html"
    style="width:1000px; height: 680px; border: 0px;""></iframe>
    """
    # colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00", "#0072B2", "#CC79A7"]
    colors= ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a", "#ffff99", "#b15928"]
    # raw_symbols = SymbolValidator().values
    # namestems = []
    # namevariants = []
    # symbols = []
    # for i in range(0,len(raw_symbols),3):
    #   name = raw_symbols[i+2]
    #   symbols.append(raw_symbols[i])
    #   namestems.append(name.replace("-open", "").replace("-dot", ""))
    #   namevariants.append(name[len(namestems[-1]):])
    # print(symbols)
    symbols = [0, 1, 2, 3, 101, 102, 103,114,115,117,126,116]

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        selected_topics = list(topics)
    elif top_n_topics is not None:
        selected_topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        selected_topics = sorted(freq_df.Topic.to_list())

    # Prepare data
    if isinstance(custom_labels, str):
        topic_names = [[[str(topic), None]] + topic_model.topic_aspects_[custom_labels][topic] for topic in topics]
        topic_names = ["_".join([label[0] for label in labels[:4]]) for labels in topic_names]
        topic_names = [label if len(label) < 30 else label[:27] + "..." for label in topic_names]
        topic_names = {key: topic_names[index] for index, key in enumerate(topic_model.topic_labels_.keys())}
    elif topic_model.custom_labels_ is not None and custom_labels:
        topic_names = {key: topic_model.custom_labels_[key + topic_model._outliers] for key, _ in topic_model.topic_labels_.items()}
    else:
        topic_names = {key: value[:40] + "..." if len(value) > 40 else value
                       for key, value in topic_model.topic_labels_.items()}
    topics_over_time["Name"] = topics_over_time.Topic.map(topic_names)
    data = topics_over_time.loc[topics_over_time.Topic.isin(selected_topics), :].sort_values(["Topic", "Timestamp"])

    # Add traces
    fig = go.Figure()
    for index, topic in enumerate(data.Topic.unique()):
        trace_data = data.loc[data.Topic == topic, :]
        topic_name = trace_data.Name.values[0]
        words = trace_data.Words.values
        if normalize_frequency:
            y = normalize(trace_data.Frequency.values.reshape(1, -1))[0]
        else:
            y = trace_data.Frequency
        fig.add_trace(go.Scatter(x=trace_data.Timestamp, y=y,
                                 marker_symbol=symbols[index % 12],
                                 mode='lines+markers',
                                 marker_color=colors[index % 12],
                                 hoverinfo="text",
                                 name=topic_name,
                                 hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))

    # Styling of the visualization
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)
    fig.update_layout(
        yaxis_title="Normalized Frequency" if normalize_frequency else "Frequency",
        title={
            'text': f"{title}",
            'y': .95,
            'x': 0.40,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        template="simple_white",
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
        legend=dict(
            title="<b>Topics",
        )
    )
    return fig

In [ ]:
topic_model.set_topic_labels({0: "Topic 0", 1: "Topic 1", 2: "Topic 2", 3: "Topic 3", 4: "Topic 4", 5: "Topic 5", 6: "Topic 6", 7: "Topic 7", 8: "Topic 8", 9: "Topic 9", 10: "Topic 10", 11: "Topic 11"})


In [ ]:
topics_over_time = topic_model.topics_over_time(data, year_month, datetime_format="%Y-%M", nr_bins=96)

topics_over_time.

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=12, normalize_frequency=True, custom_labels=True)

In [ ]:
visualize_topics_over_time(topic_model,topics_over_time, top_n_topics=12,normalize_frequency=True,custom_labels=True)

In [ ]:
ww = pd.DataFrame(topics_over_time.groupby(["Topic"])["Frequency"].max().reset_index(name='count'))

xx = pd.merge(topics_over_time, ww, on='Topic')

xx['Frequency'] = xx['Frequency'] / xx['count'] * 100.0

,Topic,Words,Frequency,Timestamp,Name,count
0,-1,"restor landscap, privat sector, landscap resto...",6.391810,2014-12-29 10:38:54.540,-1_landscap approach_restor effort_lands...,3614
1,-1,"restor plan, restor mechan, landscap restor me...",8.743774,2015-12-01 20:19:35.000,-1_landscap approach_restor effort_lands...,3614
2,-1,"work forest, resili forest landscap, restor me...",9.463199,2016-12-31 13:47:00.000,-1_landscap approach_restor effort_lands...,3614
3,-1,"via globallf, financ forest landscap, financ f...",18.649696,2017-12-31 20:40:00.000,-1_landscap approach_restor effort_lands...,3614
4,-1,"project forest, global partnership forest, glo...",12.396237,2018-12-01 16:58:35.000,-1_landscap approach_restor effort_lands...,3614
...,...,...,...,...,...,...
1323,237,"around world, , , ,",100.000000,2020-12-31 17:19:00.000,237_around world___,24
1324,246,"learn forest landscap, learn forest, landscap ...",100.000000,2020-12-31 17:19:00.000,246_learn forest landscap_learn forest_l...,23
1325,251,"digit forum, thinklandscap globallf, conserv r...",100.000000,2020-12-31 17:19:00.000,251_digit forum_thinklandscap globallf_c...,22
1326,90,"potenti forest landscap, potenti forest, case ...",100.000000,2021-12-01 13:37:35.000,90_potenti forest landscap_potenti fores...,52


In [ ]:
topic_model.visualize_topics_over_time(xx, top_n_topics=12)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,13370,-1_land use_restor effort_restor project_lands...
1,0,1085,0_habitat restor_restor landscap__
2,1,909,1_forest landscap restor_forest landscap_lands...
3,2,656,2____
4,3,389,3_forest landscap restor_forest landscap_lands...
...,...,...,...
1358,1357,20,1357_restor program_degrad land_forest landsca...
1359,1358,20,1358_landscap restor africa_look forward_resto...
1360,1359,20,1359_degrad land_million hectar_landscap restor_
1361,1360,20,1360_climat chang_forest landscap restor_fores...


In [ ]:
topic_model.get_topic(0)

[('habitat restor', 0.02985961735081114),
 ('restor landscap', 0.02214384010055814),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# fig = topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)

# fig.write_html("file.html")

fig = topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)
# fig.write_image("forest_landscape_topic.png")
# fig.write_image("landscape_topic.png")
# fig.write_image("ecosystem_topic.png")
fig.write_image("ecological_topic.png")

In [ ]:
topic_model.visualize_barchart(top_n_topics=12, width=512, height=512, n_words=20)

In [ ]:
# topic_model.reduce_topics(data, nr_topics=10)
# from bertopic import BERTopic
topic_model.reduce_topics(data, nr_topics="auto")


In [ ]:
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
topic_model.visualize_hierarchy()